In [ ]:
import pandas as pd
import re
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
from collections import defaultdict
import numpy as np
from datetime import datetime

class AnalizadorCitasLocales:
    """Clase para analizar citas locales en archivos de bibliometrix."""

    def __init__(self, archivo_excel, metodo='exacto', output_dir='resultados_citas'):
        """
        Inicializa el analizador de citas locales.

        Args:
            archivo_excel: Ruta al archivo Excel de Bibliometrix.
            metodo: Método de búsqueda de DOI ('exacto' o 'patron').
            output_dir: Directorio para guardar los resultados.
        """
        self.archivo_excel = archivo_excel
        self.metodo = metodo
        self.output_dir = output_dir
        self.citas_locales = {}
        self.info_articulos = {}
        self.red_citas = defaultdict(list)

        # Crear directorio de salida si no existe
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Leer el archivo Excel
        print(f"Leyendo archivo: {archivo_excel}")
        self.df = pd.read_excel(archivo_excel)

        # Verificar columnas necesarias
        if 'DI' not in self.df.columns or 'CR_raw' not in self.df.columns:
            raise ValueError("El archivo no contiene las columnas requeridas (DI y/o CR_raw)")

        # Definir patrones para búsqueda de DOIs
        if metodo == 'patron':
            self.patrones_doi = [
                r'DOI\s+([^\s;,]+)',
                r'DOI[:/]?\s*([^\s;,]+)',
                r',\s*DOI\s+([^\s;,]+)',
                r'[,;]\s*([0-9]+\.[0-9]+/[^\s;,]+)',
                r'10\.[0-9]{4,}[/.-][^\s;,)]+',
            ]
        else:
            self.patrones_doi = None

    def extraer_info_articulos(self):
        """Extrae la información básica de todos los artículos con DOI."""
        print("Extrayendo información de artículos...")

        for idx, row in self.df.iterrows():
            # Obtener el DOI de la fila actual
            doi = str(row['DI']).strip() if not pd.isna(row['DI']) else None

            # Si no hay DOI, continuar con la siguiente fila
            if not doi or doi == 'nan' or doi.lower() == 'none':
                continue

            # Normalizar el DOI
            doi_norm = doi.lower()

            # Inicializar contador para este DOI
            if doi_norm not in self.citas_locales:
                self.citas_locales[doi_norm] = 0

            # Guardar información del artículo
            self.info_articulos[doi_norm] = {
                'Titulo': row.get('TI', "N/A") if not pd.isna(row.get('TI', "N/A")) else "N/A",
                'Autores': row.get('AU', "N/A") if not pd.isna(row.get('AU', "N/A")) else "N/A",
                'Año': row.get('PY', "N/A") if not pd.isna(row.get('PY', "N/A")) else "N/A",
                'Revista': row.get('SO', "N/A") if not pd.isna(row.get('SO', "N/A")) else "N/A",
                'TC': row.get('TC', 0) if not pd.isna(row.get('TC', 0)) else 0,  # Citas totales (WoS)
                'Indice': idx  # Guardar el índice original para referencia
            }

        print(f"Se encontraron {len(self.citas_locales)} artículos con DOI.")

    def contar_citas_locales(self):
        """Cuenta las citas locales para cada DOI."""
        print("Buscando citas locales...")

        # Lista de todos los DOIs normalizados
        todos_dois = list(self.citas_locales.keys())

        # Para cada fila, buscar citas a otros DOIs
        for idx, row in tqdm(self.df.iterrows(), total=len(self.df)):
            # Obtener el DOI de la fila actual (citante)
            doi_citante = str(row['DI']).strip().lower() if not pd.isna(row['DI']) else None

            # Obtener las referencias de la fila actual
            refs = str(row['CR_raw']) if not pd.isna(row['CR_raw']) else ""
            refs_lower = refs.lower()

            # Método de búsqueda: exacto o por patrones
            if self.metodo == 'patron':
                # Extraer DOIs por patrones
                dois_encontrados = set()
                for patron in self.patrones_doi:
                    matches = re.findall(patron, refs, re.IGNORECASE)
                    dois_encontrados.update([m.lower() for m in matches if m])

                # Contar citas para cada DOI encontrado
                for doi in dois_encontrados:
                    # Solo considerar DOIs que están en nuestro conjunto de artículos
                    doi_limpio = self._limpiar_doi(doi)
                    if doi_limpio in todos_dois:
                        self.citas_locales[doi_limpio] += 1

                        # Registrar la relación de citación para análisis de red
                        if doi_citante and doi_citante != doi_limpio:
                            self.red_citas[doi_limpio].append(doi_citante)
            else:
                # Método exacto: buscar cada DOI directamente en las referencias
                for doi in todos_dois:
                    if doi in refs_lower:
                        self.citas_locales[doi] += 1

                        # Registrar la relación de citación para análisis de red
                        if doi_citante and doi_citante != doi:
                            self.red_citas[doi].append(doi_citante)

        print("Conteo de citas locales completado.")

    def generar_resultados(self):
        """Genera un DataFrame con los resultados."""
        print("Generando resultados...")

        # Crear un DataFrame con los resultados
        resultados = pd.DataFrame({
            'DOI': list(self.citas_locales.keys()),
            'Citas_Locales': list(self.citas_locales.values())
        })

        # Filtrar DOIs con 0 citas
        resultados = resultados[resultados['Citas_Locales'] > 0]

        # Ordenar por número de citas (de mayor a menor)
        resultados = resultados.sort_values(by='Citas_Locales', ascending=False).reset_index(drop=True)

        # Agregar información adicional de los artículos
        resultados['Titulo'] = resultados['DOI'].apply(
            lambda x: self.info_articulos.get(x, {}).get('Titulo', "N/A")
        )
        resultados['Autores'] = resultados['DOI'].apply(
            lambda x: self.info_articulos.get(x, {}).get('Autores', "N/A")
        )
        resultados['Año'] = resultados['DOI'].apply(
            lambda x: self.info_articulos.get(x, {}).get('Año', "N/A")
        )
        resultados['Revista'] = resultados['DOI'].apply(
            lambda x: self.info_articulos.get(x, {}).get('Revista', "N/A")
        )
        resultados['Citas_Totales'] = resultados['DOI'].apply(
            lambda x: self.info_articulos.get(x, {}).get('TC', 0)
        )

        # Calcular índice de localidad (% de citas que son locales)
        resultados['Indice_Localidad'] = resultados.apply(
            lambda row: round((row['Citas_Locales'] / row['Citas_Totales']) * 100, 2)
            if row['Citas_Totales'] > 0 else 0,
            axis=1
        )

        # Reordenar las columnas para mejor visualización
        resultados = resultados[[
            'DOI', 'Citas_Locales', 'Citas_Totales', 'Indice_Localidad',
            'Titulo', 'Autores', 'Año', 'Revista'
        ]]

        print(f"Se encontraron {len(resultados)} artículos con al menos una cita local.")

        return resultados

    def analizar_redes_citacion(self, resultados):
        """
        Analiza las redes de citación entre artículos.

        Args:
            resultados: DataFrame con los resultados de citas locales

        Returns:
            DataFrame con métricas de red de citas
        """
        print("Analizando redes de citación...")

        # Inicializar métricas de red
        metricas_red = {
            'DOI': [],
            'Grado_Entrada': [],  # Número de artículos que citan a este
            'Grado_Salida': [],   # Número de artículos que este cita
            'Centralidad': []     # Medida de la importancia del nodo en la red
        }

        # Calcular grado de entrada para cada artículo
        for doi in resultados['DOI']:
            # Grado de entrada (artículos que citan a este)
            grado_entrada = len(self.red_citas.get(doi, []))

            # Grado de salida (artículos que este cita)
            grado_salida = sum(1 for citante_list in self.red_citas.values() if doi in citante_list)

            # Centralidad simple (entrada + salida)
            centralidad = grado_entrada + grado_salida

            # Guardar métricas
            metricas_red['DOI'].append(doi)
            metricas_red['Grado_Entrada'].append(grado_entrada)
            metricas_red['Grado_Salida'].append(grado_salida)
            metricas_red['Centralidad'].append(centralidad)

        # Crear DataFrame de métricas de red
        df_metricas = pd.DataFrame(metricas_red)

        # Combinar con resultados originales
        resultados_completos = pd.merge(resultados, df_metricas, on='DOI', how='left')

        print("Análisis de redes completado.")
        return resultados_completos

    def generar_estadisticas(self, resultados):
        """
        Genera estadísticas descriptivas sobre las citas locales.

        Args:
            resultados: DataFrame con los resultados de citas locales

        Returns:
            DataFrame con estadísticas
        """
        print("Generando estadísticas descriptivas...")

        # Estadísticas básicas de citas locales
        estadisticas = {
            'Métrica': [
                'Total de artículos analizados',
                'Artículos con DOI',
                'Artículos con al menos una cita local',
                'Promedio de citas locales por artículo',
                'Mediana de citas locales',
                'Máximo de citas locales',
                'Porcentaje de artículos con citas locales',
                'Índice de localidad promedio'
            ],
            'Valor': [
                len(self.df),
                len(self.citas_locales),
                len(resultados),
                round(resultados['Citas_Locales'].mean(), 2),
                resultados['Citas_Locales'].median(),
                resultados['Citas_Locales'].max(),
                round(len(resultados) / len(self.citas_locales) * 100, 2) if len(self.citas_locales) > 0 else 0,
                round(resultados['Indice_Localidad'].mean(), 2) if 'Indice_Localidad' in resultados.columns else 0
            ]
        }

        # Distribución por años
        if len(resultados) > 0 and 'Año' in resultados.columns:
            # Contar artículos por año
            conteo_anios = resultados['Año'].value_counts().sort_index()
            estadisticas['Métrica'].append('Distribución por años')
            estadisticas['Valor'].append(str(dict(conteo_anios)))

        # Crear DataFrame de estadísticas
        df_estadisticas = pd.DataFrame(estadisticas)

        print("Estadísticas descriptivas generadas.")
        return df_estadisticas

    def visualizar_resultados(self, resultados):
        """
        Genera visualizaciones de los resultados.

        Args:
            resultados: DataFrame con los resultados
        """
        print("Generando visualizaciones...")

        # Configuración de estilo
        plt.style.use('seaborn-v0_8-whitegrid')

        # 1. Distribución de citas locales
        plt.figure(figsize=(12, 6))
        sns.histplot(resultados['Citas_Locales'], kde=True, bins=20)
        plt.title('Distribución de Citas Locales')
        plt.xlabel('Número de Citas Locales')
        plt.ylabel('Frecuencia')
        plt.savefig(os.path.join(self.output_dir, 'distribucion_citas_locales.png'), dpi=300, bbox_inches='tight')

        # 2. Top 20 artículos más citados localmente
        top_20 = resultados.head(20)
        plt.figure(figsize=(14, 8))
        barplot = sns.barplot(x='Citas_Locales', y='Titulo', data=top_20)
        plt.title('Top 20 Artículos Más Citados Localmente')
        plt.xlabel('Número de Citas Locales')
        plt.ylabel('Título del Artículo')
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, 'top20_citas_locales.png'), dpi=300, bbox_inches='tight')

        # 3. Citas locales vs. citas totales
        if 'Citas_Totales' in resultados.columns:
            plt.figure(figsize=(10, 8))
            scatter = plt.scatter(
                resultados['Citas_Totales'],
                resultados['Citas_Locales'],
                alpha=0.6,
                c=resultados['Indice_Localidad'] if 'Indice_Localidad' in resultados.columns else 'blue',
                cmap='viridis'
            )
            plt.colorbar(scatter, label='Índice de Localidad (%)' if 'Indice_Localidad' in resultados.columns else '')
            plt.title('Citas Locales vs. Citas Totales')
            plt.xlabel('Citas Totales')
            plt.ylabel('Citas Locales')
            plt.grid(True, alpha=0.3)

            # Añadir línea de tendencia
            z = np.polyfit(resultados['Citas_Totales'], resultados['Citas_Locales'], 1)
            p = np.poly1d(z)
            plt.plot(resultados['Citas_Totales'], p(resultados['Citas_Totales']), "r--", alpha=0.8)

            plt.savefig(os.path.join(self.output_dir, 'citas_locales_vs_totales.png'), dpi=300, bbox_inches='tight')

        # 4. Distribución de citas por año de publicación
        if 'Año' in resultados.columns:
            plt.figure(figsize=(12, 6))
            # Convertir año a valor numérico si es necesario
            resultados['Año_Num'] = pd.to_numeric(resultados['Año'], errors='coerce')
            # Agrupar por año y calcular promedio de citas
            citas_por_anio = resultados.groupby('Año_Num')['Citas_Locales'].agg(['mean', 'count']).reset_index()

            # Crear gráfico de barras con línea
            fig, ax1 = plt.subplots(figsize=(12, 6))
            ax1.bar(citas_por_anio['Año_Num'], citas_por_anio['count'], color='skyblue', alpha=0.7)
            ax1.set_xlabel('Año de Publicación')
            ax1.set_ylabel('Número de Artículos', color='skyblue')
            ax1.tick_params(axis='y', labelcolor='skyblue')

            ax2 = ax1.twinx()
            ax2.plot(citas_por_anio['Año_Num'], citas_por_anio['mean'], 'r-', linewidth=2)
            ax2.set_ylabel('Promedio de Citas Locales', color='red')
            ax2.tick_params(axis='y', labelcolor='red')

            plt.title('Distribución de Artículos y Citas Locales por Año')
            plt.tight_layout()
            plt.savefig(os.path.join(self.output_dir, 'citas_por_anio.png'), dpi=300, bbox_inches='tight')

        plt.close('all')
        print("Visualizaciones guardadas en el directorio:", self.output_dir)

    def generar_informe_html(self, resultados, estadisticas):
        """
        Genera un informe HTML con los resultados y estadísticas.

        Args:
            resultados: DataFrame con los resultados
            estadisticas: DataFrame con las estadísticas
        """
        print("Generando informe HTML...")

        # Obtener fecha y hora actual
        fecha_actual = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

        # Cabecera del informe
        cabecera = f"""
        <!DOCTYPE html>
        <html lang="es">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Análisis de Citas Locales</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 20px; line-height: 1.6; }}
                h1, h2, h3 {{ color: #2c3e50; }}
                table {{ border-collapse: collapse; width: 100%; margin-bottom: 20px; }}
                th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
                th {{ background-color: #f2f2f2; }}
                tr:nth-child(even) {{ background-color: #f9f9f9; }}
                .container {{ max-width: 1200px; margin: 0 auto; }}
                .statistics {{ display: flex; flex-wrap: wrap; margin-bottom: 20px; }}
                .stat-box {{ flex: 1; min-width: 250px; margin: 10px; padding: 15px; border-radius: 5px; background-color: #f8f9fa; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }}
                .stat-value {{ font-size: 24px; font-weight: bold; color: #3498db; }}
                .stat-label {{ font-size: 14px; color: #7f8c8d; }}
                .img-container {{ margin: 20px 0; text-align: center; }}
                .img-container img {{ max-width: 100%; height: auto; border: 1px solid #ddd; border-radius: 4px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1>Análisis de Citas Locales</h1>
                <p><strong>Fecha del análisis:</strong> {fecha_actual}</p>
                <p><strong>Archivo analizado:</strong> {os.path.basename(self.archivo_excel)}</p>
                <p><strong>Método de búsqueda:</strong> {self.metodo}</p>
        """

        # Sección de estadísticas
        estadisticas_html = """
                <h2>Estadísticas Generales</h2>
                <div class="statistics">
        """

        for _, row in estadisticas.iterrows():
            metrica = row['Métrica']
            valor = row['Valor']

            # No incluir la distribución por años en los cuadros de estadísticas
            if metrica != 'Distribución por años':
                estadisticas_html += f"""
                    <div class="stat-box">
                        <div class="stat-value">{valor}</div>
                        <div class="stat-label">{metrica}</div>
                    </div>
                """

        estadisticas_html += """
                </div>
        """

        # Sección de visualizaciones
        visualizaciones_html = """
                <h2>Visualizaciones</h2>

                <h3>Distribución de Citas Locales</h3>
                <div class="img-container">
                    <img src="distribucion_citas_locales.png" alt="Distribución de Citas Locales">
                </div>

                <h3>Top 20 Artículos Más Citados Localmente</h3>
                <div class="img-container">
                    <img src="top20_citas_locales.png" alt="Top 20 Artículos Más Citados Localmente">
                </div>
        """

        # Añadir visualizaciones condicionales
        if 'Citas_Totales' in resultados.columns:
            visualizaciones_html += """
                <h3>Citas Locales vs. Citas Totales</h3>
                <div class="img-container">
                    <img src="citas_locales_vs_totales.png" alt="Citas Locales vs. Citas Totales">
                </div>
            """

        if 'Año' in resultados.columns:
            visualizaciones_html += """
                <h3>Distribución de Artículos y Citas por Año</h3>
                <div class="img-container">
                    <img src="citas_por_anio.png" alt="Distribución de Citas por Año">
                </div>
            """

        # Sección de resultados (tabla de DOIs con citas)
        resultados_html = """
                <h2>Artículos con Citas Locales</h2>
                <table>
                    <tr>
                        <th>#</th>
                        <th>DOI</th>
                        <th>Citas Locales</th>
        """

        # Añadir encabezados adicionales según columnas disponibles
        if 'Citas_Totales' in resultados.columns:
            resultados_html += """
                        <th>Citas Totales</th>
                        <th>Índice de Localidad</th>
            """

        resultados_html += """
                        <th>Título</th>
                        <th>Autores</th>
                        <th>Año</th>
                        <th>Revista</th>
                    </tr>
        """

        # Añadir filas de datos (limitadas a los primeros 100 para no hacer el HTML demasiado grande)
        for idx, row in resultados.head(100).iterrows():
            resultados_html += f"""
                    <tr>
                        <td>{idx + 1}</td>
                        <td><a href="https://doi.org/{row['DOI']}" target="_blank">{row['DOI']}</a></td>
                        <td>{row['Citas_Locales']}</td>
            """

            if 'Citas_Totales' in resultados.columns:
                resultados_html += f"""
                        <td>{row['Citas_Totales']}</td>
                        <td>{row['Indice_Localidad']}%</td>
                """

            resultados_html += f"""
                        <td>{row['Titulo']}</td>
                        <td>{row['Autores']}</td>
                        <td>{row['Año']}</td>
                        <td>{row['Revista']}</td>
                    </tr>
            """

        # Si hay más de 100 resultados, indicarlo
        if len(resultados) > 100:
            resultados_html += f"""
                    <tr>
                        <td colspan="8" style="text-align: center; font-style: italic;">
                            Mostrando 100 de {len(resultados)} resultados. Ver archivo Excel para el conjunto completo.
                        </td>
                    </tr>
            """

        # Cerrar tabla y documento
        resultados_html += """
                </table>
            </div>
        </body>
        </html>
        """

        # Combinar todas las secciones
        informe_html = cabecera + estadisticas_html + visualizaciones_html + resultados_html

        # Guardar informe HTML
        ruta_informe = os.path.join(self.output_dir, "informe_citas_locales.html")
        with open(ruta_informe, 'w', encoding='utf-8') as f:
            f.write(informe_html)

        print(f"Informe HTML generado en: {ruta_informe}")

    def ejecutar_analisis_completo(self):
        """Ejecuta el proceso completo de análisis de citas locales."""
        # 1. Extraer información de artículos
        self.extraer_info_articulos()

        # 2. Contar citas locales
        self.contar_citas_locales()

        # 3. Generar resultados básicos
        resultados = self.generar_resultados()

        # 4. Analizar redes de citación
        resultados_completos = self.analizar_redes_citacion(resultados)

        # 5. Generar estadísticas
        estadisticas = self.generar_estadisticas(resultados_completos)

        # 6. Visualizar resultados
        self.visualizar_resultados(resultados_completos)

        # 7. Generar informe HTML
        self.generar_informe_html(resultados_completos, estadisticas)

        # 8. Guardar resultados en Excel
        ruta_excel = os.path.join(self.output_dir, "citas_locales_resultados.xlsx")
        resultados_completos.to_excel(ruta_excel, index=False)
        print(f"Resultados guardados en: {ruta_excel}")

        # 9. Guardar estadísticas en Excel
        ruta_estadisticas = os.path.join(self.output_dir, "citas_locales_estadisticas.xlsx")
        estadisticas.to_excel(ruta_estadisticas, index=False)
        print(f"Estadísticas guardadas en: {ruta_estadisticas}")

        return resultados_completos, estadisticas

    def _limpiar_doi(self, doi):
        """
        Limpia un DOI para estandarizarlo.

        Args:
            doi: DOI a limpiar

        Returns:
            DOI limpio
        """
        # Eliminar caracteres no deseados al final
        doi = re.sub(r'[,;.]$', '', doi)

        # Eliminar texto dentro de corchetes o paréntesis
        doi = re.sub(r'\([^)]*\)', '', doi)
        doi = re.sub(r'\[[^\]]*\]', '', doi)

        # Eliminar cualquier texto que no sea parte del DOI estándar
        if match := re.search(r'(10\.\d{4,}/[^,;\s]+)', doi):
            return match.group(1).lower()

        return doi.lower()


if __name__ == "__main__":
    # Archivo de entrada (ajustar la ruta según sea necesario)
    archivo_entrada = "BibliometrixExportFile20250220.xlsx"

    # Crear instancia del analizador y ejecutar análisis completo
    analizador = AnalizadorCitasLocales(
        archivo_excel=archivo_entrada,
        metodo='patron',  # Usar 'patron' para búsqueda más exhaustiva
        output_dir='resultados_citas_locales'
    )

    # Ejecutar análisis completo
    resultados, estadisticas = analizador.ejecutar_analisis_completo()

    # Mostrar los 10 artículos más citados localmente
    print("\nTop 15 artículos más citados localmente:")
    print(resultados.head(15)[['DOI', 'Citas_Locales', 'Titulo', 'Autores', 'Año']])

    print("\nEstadísticas principales:")
    print(estadisticas)

    print(f"\nAnálisis completado. Todos los resultados están disponibles en la carpeta '{analizador.output_dir}'.")

Leyendo archivo: BibliometrixExportFile20250220.xlsx
Extrayendo información de artículos...
Se encontraron 359 artículos con DOI.
Buscando citas locales...


100%|██████████| 441/441 [00:00<00:00, 2348.38it/s]


Conteo de citas locales completado.
Generando resultados...
Se encontraron 14 artículos con al menos una cita local.
Analizando redes de citación...
Análisis de redes completado.
Generando estadísticas descriptivas...
Estadísticas descriptivas generadas.
Generando visualizaciones...
Visualizaciones guardadas en el directorio: resultados_citas_locales
Generando informe HTML...
Informe HTML generado en: resultados_citas_locales/informe_citas_locales.html
Resultados guardados en: resultados_citas_locales/citas_locales_resultados.xlsx
Estadísticas guardadas en: resultados_citas_locales/citas_locales_estadisticas.xlsx

Top 10 artículos más citados localmente:
                              DOI  Citas_Locales  \
0    10.1007/978-3-031-05897-4\_6              2   
1    10.1007/978-3-030-22646-6\_6              1   
2     10.1109/access.2021.3069559              1   
3    10.1109/eurocon.2019.8861847              1   
4  10.1016/j.ijmedinf.2022.104735              1   
5  10.1016/j.ijmedinf.202